In [6]:
import pandas as pd
import networkx as nx
#get known complexes
datadir = '/home/dmoi/datasets/foldtree2/complexes/'
complexdf = pd.read_csv(datadir+'contactDefinition.txt' , sep='\t')
print(complexdf.head())

     code ch1 ch2  nres  len1  len2 dom_s1      dom_p1 dom_s2      dom_p2  \
0  4rk1_1   A   B  31.0   267   268  53822   PF13377.1  53822   PF13377.1   
1  2gqn_1   D   B  53.0   392   392  53383  PF01053.15  53383  PF01053.15   
2  2gqn_1   D   C  17.0   392   391  53383  PF01053.15  53383  PF01053.15   
3  2gqn_1   A   C  52.0   391   391  53383  PF01053.15  53383  PF01053.15   
4  2gqn_1   A   D  28.5   391   392  53383  PF01053.15  53383  PF01053.15   

   ident  homo  
0      1   1.0  
1      1   1.0  
2      1   1.0  
3      1   1.0  
4      1   1.0  


In [7]:
import glob
pdbs = glob.glob(datadir+'BU_all_renum/*.pdb')
print(len(pdbs))

173593


In [8]:
codes = { f.split('/')[-1].split('.')[0]:f for f in pdbs}
complexdf['pdbfile'] = complexdf['code'].map(codes)
print(complexdf.head())

     code ch1 ch2  nres  len1  len2 dom_s1      dom_p1 dom_s2      dom_p2  \
0  4rk1_1   A   B  31.0   267   268  53822   PF13377.1  53822   PF13377.1   
1  2gqn_1   D   B  53.0   392   392  53383  PF01053.15  53383  PF01053.15   
2  2gqn_1   D   C  17.0   392   391  53383  PF01053.15  53383  PF01053.15   
3  2gqn_1   A   C  52.0   391   391  53383  PF01053.15  53383  PF01053.15   
4  2gqn_1   A   D  28.5   391   392  53383  PF01053.15  53383  PF01053.15   

   ident  homo                                            pdbfile  
0      1   1.0  /home/dmoi/datasets/foldtree2/complexes/BU_all...  
1      1   1.0  /home/dmoi/datasets/foldtree2/complexes/BU_all...  
2      1   1.0  /home/dmoi/datasets/foldtree2/complexes/BU_all...  
3      1   1.0  /home/dmoi/datasets/foldtree2/complexes/BU_all...  
4      1   1.0  /home/dmoi/datasets/foldtree2/complexes/BU_all...  


In [9]:
#sample 2000 codes
import random
sub = complexdf.code.unique().tolist()
random.shuffle(sub)
sub = sub[:2000]
sub = complexdf[complexdf.code.isin(sub)]
print(sub.head())
print( len(sub) )

       code ch1 ch2  nres  len1  len2                   dom_s1  \
89   3wgo_1   A   B  56.0   323   324  51735_f1;55347;51735_f2   
388  4e8y_1   A   B  22.0   308   308                      NaN   
570  4qar_2   A   B  15.0   195   187                      NaN   
895  3wy0_1   A   B  49.5   372   377                      NaN   
897  4n8i_3   A   B  48.0   440   440                      NaN   

                    dom_p1                   dom_s2                 dom_p2  \
89              PF01113.15  51735_f1;55347;51735_f2             PF01113.15   
388             PF00294.19                      NaN             PF00294.19   
570                    NaN                      NaN                    NaN   
895  PF00195.14;PF02797.10                      NaN  PF00195.14;PF02797.10   
897   PF02550.10;PF13336.1                      NaN   PF02550.10;PF13336.1   

     ident  homo                                            pdbfile  
89       1   1.0  /home/dmoi/datasets/foldtree2/complexes/BU_all

In [6]:
import tqdm
import pickle
chunksize = 10000
graphs = {}
for code in tqdm.tqdm(complexdf.code.unique()):
    sub = complexdf[complexdf.code == code]
    chains = set(sub.ch1.unique()).union( set( sub.ch2.unique() ))
    #add chains to graph
    G = nx.MultiGraph()
    G.add_nodes_from(chains)
    for i, row in sub.iterrows():
        #homology link
        if row.homo == 1:
            #add homology type edge
            ekey = G.add_edge(row.ch1, row.ch2, key='homology' )
        #add contact type edge
        ekey = G.add_edge(row.ch1, row.ch2, key='contact' )
    graphs[code] = G

with open('complexgraphs.pkl' , 'wb') as graphsout:
    graphsout.write( pickle.dumps( graphs ) )

#embed all structures
#use decoder and generate z vecs
#use decoder sigmoid to get contact proba

 40%|█████████████████████████████▊                                            | 36180/89764 [10:45<15:56, 56.02it/s]


KeyboardInterrupt: 

In [ ]:
from torch_geometric.data import HeteroData
#import sageconv
from torch_geometric.nn import SAGEConv , Linear , FiLMConv , TransformerConv , FeaStConv , GATConv , GINConv , GatedGraphConv
#import module dict and module list
from torch.nn import ModuleDict, ModuleList , L1Loss
from torch_geometric.nn import global_mean_pool
#import negative sampling
from torch_geometric.utils import negative_sampling

EPS = 1e-10
def recon_loss_complex( z1: Tensor,z2: Tensor, pos_edge_index: Tensor , backbone:Tensor = None , decoder = None , poslossmod = 1 , neglossmod= 1) -> Tensor:
    r"""Given latent variables :obj:`z`, computes the binary cross
    entropy loss for positive edges :obj:`pos_edge_index` and negative
    sampled edges.

    Args:
        z (torch.Tensor): The latent space :math:`\mathbf{Z}`.
        pos_edge_index (torch.Tensor): The positive edges to train against.
        neg_edge_index (torch.Tensor, optional): The negative edges to
            train against. If not given, uses negative sampling to
            calculate negative edges. (default: :obj:`None`)
    """
    
    pos =decoder(z1,z2, pos_edge_index, { ( 'res','backbone','res'): backbone } )[1]
    #turn pos edge index into a binary matrix
    pos_loss = -torch.log( pos + EPS).mean()
    neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
    neg = decoder(z1 , z2, neg_edge_index, { ( 'res','backbone','res'): backbone } )[1]
    neg_loss = -torch.log( ( 1 - neg) + EPS ).mean()
    return poslossmod*pos_loss + neglossmod*neg_loss


In [34]:
from torch_geometric.nn import SAGEConv , Linear
from torch.nn import Sigmoid, ModuleDict

class HeteroGAE_Pairwise_Decoder(torch.nn.Module):
    #we don't need to decode to aa... just contact probs
    def __init__(self, encoder_out_channels, xdim=20, hidden_channels={'res_backbone_res': [20, 20, 20]}, out_channels_hidden=20, nheads = 1 , Xdecoder_hidden=30, metadata={}, amino_mapper= None):
        super(HeteroGAE_Pairwise_Decoder, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.metadata = metadata
        self.hidden_channels = hidden_channels
        self.out_channels_hidden = out_channels_hidden
        self.in_channels = encoder_out_channels
        for i in range(len(self.hidden_channels[('res', 'backbone', 'res')])):
            self.convs.append(
                torch.nn.ModuleDict({
                    '_'.join(edge_type): SAGEConv(self.in_channels if i == 0 else self.hidden_channels[edge_type][i-1], self.hidden_channels[edge_type][i]  )
                    for edge_type in [('res', 'backbone', 'res')]
                })
            )
        self.lin = Linear(hidden_channels[('res', 'backbone', 'res')][-1], self.out_channels_hidden)
        self.sigmoid = Sigmoid()
    def forward(self, z1, z2, edge_index, backbones, **kwargs):
        zs = []
        for z in [z1,z2]:
            inz = z
            for layer in self.convs:
                for edge_type, conv in layer.items():
                    z = conv(z, backbones[tuple(edge_type.split('_'))])
                    z = F.relu(z)
            z = self.lin(z)
            zs.append(z)
        sim_matrix = (zs[0][edge_index[0]] * zs[1][edge_index[1]]).sum(dim=1)
        edge_probs = self.sigmoid(sim_matrix)
        
        return edge_probs

In [35]:
cd projects/foldtree2/

[Errno 2] No such file or directory: 'projects/foldtree2/'
/home/dmoi/projects/foldtree2


In [36]:
import foldtree2_ecddcd as ft2
#reload ft2
import importlib

importlib.reload(ft2)

<module 'foldtree2_ecddcd' from '/home/dmoi/projects/foldtree2/foldtree2_ecddcd.py'>

In [40]:
import torch
import pickle
import os

ndim = 844
converter = ft2.PDB2PyG()
with open( 'model.pkl' , 'rb' ) as f:
    encoder, d = pickle.load( f )
alphabetsize = encoder.vector_quantizer.embedding_dim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
encoder.eval()

#create new decoder
decoder = HeteroGAE_Pairwise_Decoder(encoder_out_channels = encoder.out_channels , 
                            hidden_channels={ ( 'res','backbone','res'):[ 300 ] * 5  } , 
                            out_channels_hidden= 150 , metadata={} , Xdecoder_hidden=100 )

decoder_save = 'decoder_complex1st'
#put encoder and decoder on the device
encoder = encoder.to(device)
decoder = decoder.to(device)

# Create a DataLoader for training
total_loss_x = 0
total_loss_edge = 0
total_vq=0
total_kl = 0
total_plddt=0
# Training loop
if os.path.exists(decoder_save):
    decoder.load_state_dict(torch.load(decoder_save))
    print("loaded encoder and decoder")


In [ ]:
train_loader = DataLoader(struct_dat, batch_size=40, shuffle=True)
optimizer = torch.optim.Adam( list(decoder.parameters()), lr=0.001)
encoder.train()
decoder.train()
edgelosses = []
edgeweight = 1
for epoch in range(1000):
    for data in tqdm.tqdm(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        z1 = encoder(data['res'].x, data['AA'].x , data.edge_index_dict)
        z2 = encoder(data['res'].x, data['AA'].x , data.edge_index_dict)
        #add positional encoding to give to the decoder
        edgeloss = recon_loss(z1,z2, , data.edge_index_dict[( 'res','contactPoints','res')]
                                , data.edge_index_dict[( 'res','backbone','res')], decoder)
        
        loss =  edgeweight*edgeloss
        loss.backward()
        optimizer.step()
        total_loss_edge += edgeloss.item()
    if epoch % 100 == 0 :
        #save model
        torch.save(decoder.state_dict(), decoder_save)
        with open('decoder_complex.pkl' , 'wb') as out:
            out.write( pickle.dumps( decoder ) )
    
    print(f'Epoch {epoch}, Edge Loss: {total_loss_edge:.4f}') 
    total_loss_edge = 0
    #total_plddt = 0
torch.save(decoder.state_dict(), decoder_save)
with open('decoder_complex.pkl' , 'wb') as out:
    out.write( pickle.dumps( decoder ) )

In [ ]:
#brainstorming to get complex graphs...

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn import TransformerDecoder, TransformerDecoderLayer

class GraphGenerationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, nhead, num_encoder_layers, num_decoder_layers, num_classes):
        super(GraphGenerationModel, self).__init__()
        self.encoder_layer = TransformerEncoderLayer(d_model=input_dim, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)
        
        self.decoder_layer = TransformerDecoderLayer(d_model=output_dim, nhead=nhead)
        self.transformer_decoder = TransformerDecoder(self.decoder_layer, num_layers=num_decoder_layers)
        
        self.node_mlp = nn.Linear(output_dim, num_classes + 1)  # +1 for EOS token
        self.edge_mlp = nn.Linear(2 * output_dim, 1)
        self.embedding = nn.Embedding(num_classes + 1, output_dim)
        
    def encode(self, x):
        return self.transformer_encoder(x)
    
    def decode(self, z, memory):
        return self.transformer_decoder(z, memory)
    
    def forward(self, x, max_length=20):
        # Encode the input sequence using the transformer encoder
        memory = self.encode(x)  # Shape: [batch_size, seq_len, input_dim]
        
        # Start with an initial input (e.g., a start token)
        start_token = torch.zeros((memory.size(0), 1, memory.size(-1)), device=x.device)
        z = start_token
        
        node_logits = []
        for _ in range(max_length):
            z = self.decode(z, memory)
            node_logit = self.node_mlp(z[:, -1, :])  # Take the last token's output
            node_logits.append(node_logit)
            
            # Get the next input token (embedding of the predicted class)
            next_token = node_logit.argmax(dim=-1).unsqueeze(1)
            next_token_embedding = self.embedding(next_token)
            z = torch.cat([z, next_token_embedding], dim=1)
            
            if (next_token == num_classes).all():  # EOS token
                break
        
        node_logits = torch.stack(node_logits, dim=1)
        
        # Decode edges (fully connected example, customize as needed)
        num_nodes = node_logits.size(1)
        edge_index = torch.combinations(torch.arange(num_nodes), r=2).t().contiguous()
        edge_features = torch.cat([node_logits[:, edge_index[0]], node_logits[:, edge_index[1]]], dim=-1)
        edge_logits = self.edge_mlp(edge_features).squeeze(-1)
        edge_probs = torch.sigmoid(edge_logits)
        
        return node_logits, edge_index, edge_probs

# Example usage
input_dim = 10   # Dimension of input embeddings
hidden_dim = 32  # Dimension of hidden layer
output_dim = 16  # Dimension of node features
nhead = 2        # Number of heads in the multi-head attention
num_encoder_layers = 2  # Number of transformer encoder layers
num_decoder_layers = 2  # Number of transformer decoder layers
num_classes = 5  # Number of node categories (excluding EOS token)

model = GraphGenerationModel(input_dim, hidden_dim, output_dim, nhead, num_encoder_layers, num_decoder_layers, num_classes)

# Example input: batch of sequences (batch_size, seq_len, input_dim)
x = torch.randn(5, 10, input_dim)  # Batch of 5 sequences, each of length 10

node_logits, edge_index, edge_probs = model(x)
print("Node logits:\n", node_logits)
print("Edge index:\n", edge_index)
print("Edge probabilities:\n", edge_probs)


In [ ]:
import torch.optim as optim

def train(model, dataloader, epochs=10):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    edge_loss_fn = nn.BCELoss()  # Loss function for edge probabilities
    node_loss_fn = nn.NLLLoss()  # Loss function for node classification

    for epoch in range(epochs):
        model.train()
        total_edge_loss = 0
        total_node_loss = 0

        for sequences, graphs, node_labels in dataloader:
            optimizer.zero_grad()
            
            node_logits, edge_index, edge_probs = model(sequences)

            # Flatten the edge probabilities and create a target tensor
            target_edge_probs = torch.cat([graph.edge_index for graph in graphs], dim=1)
            target = torch.ones(target_edge_probs.size(1))  # Example target, customize as needed

            # Compute edge loss
            edge_loss = edge_loss_fn(edge_probs, target)

            # Compute node classification loss
            node_labels = node_labels.view(-1)
            node_classes = node_logits.view(-1, num_classes + 1)  # +1 for EOS token
            node_loss = node_loss_fn(node_classes, node_labels)

            # Total loss
            loss = edge_loss + node_loss
            loss.backward()
            optimizer.step()

            total_edge_loss += edge_loss.item()
            total_node_loss += node_loss.item()

        print(f"Epoch {epoch+1}, Edge Loss: {total_edge_loss / len(dataloader)}, Node Loss: {total_node_loss / len(dataloader)}")

# Train the model
train(model, dataloader)
